<a href="https://colab.research.google.com/github/dansjack/ad-450/blob/main/Week_3_Web_Scraping_(dj_winter_2022).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scrape Worldometers' Coronavirus state level data

In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.worldometers.info/coronavirus/country/us/#nav-yesterday'
resp = requests.get(url)
data = resp.text

soup = BeautifulSoup(data, 'html.parser')
table = soup.find(id="usa_table_countries_today")

## Load table into DataFrame, filter data 

In [2]:
import pandas as pd

df = pd.read_html(str(table))[0]

# state column should contain just the state name (no HTML)
df['USAState'] = df['USAState'].str.replace('\xa0', ' ')
df.set_index('USAState', inplace=True)

# remove whitespace char from column name
df.rename(columns={ "Tot\xa0Cases/1M pop": "Tot Cases/1M pop" }, inplace=True)

# exclude the '#', 'source', and 'projections' columns 
df.drop(columns=['#', 'Source', 'Projections'], inplace=True) 

# exclude rows for country totals
df.drop(["USA Total", "Total:"], inplace=True) 

# clean data, NewCases column contains '+', ',' which makes the column 
# impossible to sort later if not replaced
df['NewCases'] = df['NewCases'].str.replace('+', '').str.replace(',', '')
df['NewCases'] = df['NewCases'].astype(float)
df.fillna(0, inplace=True)

## Print the first few rows of data

In [3]:
df.head()

,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population
USAState,,,,,,,,,,,
California,7376878,0.0,78730.0,0.0,0.0,0.0,186699.0,1993.0,131160392.0,3319489.0,39512223.0
Texas,5864525,0.0,78270.0,0.0,0.0,0.0,202254.0,2699.0,55606051.0,1917722.0,28995881.0
Florida,5328573,0.0,63574.0,0.0,3665356.0,1599643.0,248098.0,2960.0,50280048.0,2341031.0,21477737.0
New York,4813820,23333.0,63619.0,56.0,2597185.0,2153016.0,247452.0,3270.0,95256290.0,4896599.0,19453561.0
Illinois,2773362,0.0,33446.0,0.0,1924335.0,815581.0,218861.0,2639.0,49488107.0,3905367.0,12671821.0


## DataFrame summary statistics 



In [4]:
summary_stats = df.describe()
summary_stats.drop(["count"])

,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population
mean,1.135471e+06,2223.666667,14098.031746,8.460317,4.785234e+05,1.981938e+05,176183.793651,2056.666667,1.431705e+07,2.356771e+06,5.263912e+06
std,1.455751e+06,5744.248175,18110.771931,25.790612,7.237332e+05,3.802140e+05,88553.010843,1170.107108,2.205543e+07,2.144921e+06,7.056634e+06
min,3.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.123280e+05,0.000000,1882.500000,0.000000,0.000000e+00,0.000000e+00,153210.000000,1319.500000,2.638481e+06,1.299232e+06,8.233605e+05
50%,6.813820e+05,0.000000,8736.000000,0.000000,1.788590e+05,1.996800e+04,209194.000000,2547.000000,6.787993e+06,2.236708e+06,3.155070e+06
75%,1.448960e+06,21.000000,17847.500000,0.000000,5.812195e+05,2.135100e+05,233894.000000,2993.000000,1.713720e+07,2.888398e+06,6.780696e+06
max,7.376878e+06,27681.000000,78730.000000,128.000000,3.665356e+06,2.153016e+06,308780.000000,3602.000000,1.311604e+08,1.445150e+07,3.951222e+07


## Top 5s

### New Cases

In [5]:
# Note: depending on when you look at the data, you'll see a lot of 0s
top_new_cases = df.sort_values(by=["NewCases"], ascending=False)
top_new_cases = pd.DataFrame(top_new_cases, columns = ['NewCases'])
top_new_cases.head()

,NewCases
USAState,
Arizona,27681.0
New York,23333.0
Pennsylvania,17623.0
Alabama,16506.0
Ohio,15677.0


### Total Deaths

In [6]:
top_total_deaths = df.sort_values(by=["TotalDeaths"], ascending=False)
top_total_deaths = pd.DataFrame(top_total_deaths, columns = ['TotalDeaths'])
top_total_deaths.head()

,TotalDeaths
USAState,
California,78730.0
Texas,78270.0
New York,63619.0
Florida,63574.0
Pennsylvania,39487.0


### Total Cases / 1m Population

In [7]:
top_cases_per_m = df.sort_values(by=["Tot Cases/1M pop"], ascending=False)
top_cases_per_m = pd.DataFrame(top_cases_per_m, columns = ['Tot Cases/1M pop'])
top_cases_per_m.head()

,Tot Cases/1M pop
USAState,
Rhode Island,308780.0
North Dakota,272944.0
Alaska,257827.0
Utah,257642.0
Tennessee,251062.0


### Total Deaths / 1m Population

In [8]:
top_deaths_per_m = df.sort_values(by=["Deaths/1M pop"], ascending=False)
top_deaths_per_m = pd.DataFrame(top_deaths_per_m, columns = ['Deaths/1M pop'])
top_deaths_per_m.head()

,Deaths/1M pop
USAState,
Mississippi,3602.0
Arizona,3511.0
New Jersey,3462.0
Alabama,3432.0
Louisiana,3296.0
